<br>

###   hw7pr1iris_modeler 

Here, you'll epxlore iris clasification <font color="Coral">and regression</font> via NNets

<font color="Coral"> _Regression_ </font> - estimating a floating-point value - is NNets' natural operation.

<br>

_Classification_ - estimating a discrete label - is done by creating discrete variables (we can't let the library treat a categorical variable as if it were continuous). <br> We'll start there, since it's a variation of our past workflows:


In [124]:
#
# hw7pr1iris_modeler:  NNETS! 
#
#   including _both_ clasification + regression for iris modeling
#

In [125]:
# libraries...
import numpy as np      # numpy is Python's "array" library
import pandas as pd     # Pandas is Python's "data" library ("dataframe" == spreadsheet)

### Reading in our data

previously cleaned...

In [126]:
# let's read in our flower data...
# 
# iris_cleaned.csv should be in this folder
# 
filename = 'iris_cleaned.csv' # neighborhoods
df_tidy = pd.read_csv(filename)      # encoding = "utf-8", "latin1"
print(f"{filename} : file read into a pandas dataframe.")

iris_cleaned.csv : file read into a pandas dataframe.


In [127]:
#
# different version vary on how to see all rows (adapt to suit your system!)
#
print(f"df_tidy.shape is {df_tidy.shape}\n")
df_tidy.info()  # prints column information
df_tidy

df_tidy.shape is (141, 6)

<class 'pandas.core.frame.DataFrame'>
Index: 141 entries, 0 to 140
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sepallen  141 non-null    float64
 1   sepalwid  141 non-null    float64
 2   petallen  141 non-null    float64
 3   petalwid  141 non-null    float64
 4   irisname  141 non-null    object 
 5   irisnum   141 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 7.7+ KB


,sepallen,sepalwid,petallen,petalwid,irisname,irisnum
0,4.6,3.6,1.0,0.2,setosa,0
1,4.3,3.0,1.1,0.1,setosa,0
2,5.0,3.2,1.2,0.2,setosa,0
3,5.8,4.0,1.2,0.2,setosa,0
4,4.4,3.0,1.3,0.2,setosa,0
...,...,...,...,...,...,...
136,7.9,3.8,6.4,2.0,virginica,2
137,7.6,3.0,6.6,2.1,virginica,2
138,7.7,3.8,6.7,2.2,virginica,2
139,7.7,2.8,6.7,2.0,virginica,2


In [128]:
#
# we need to make sure the categories are handed as categories!
#    scikit-learn calls these "dummy" variables:

df_tidy_cat = pd.get_dummies(data=df_tidy,prefix="is",columns=['irisname'])
df_tidy_cat

,sepallen,sepalwid,petallen,petalwid,irisnum,is_setosa,is_versicolor,is_virginica
0,4.6,3.6,1.0,0.2,0,True,False,False
1,4.3,3.0,1.1,0.1,0,True,False,False
2,5.0,3.2,1.2,0.2,0,True,False,False
3,5.8,4.0,1.2,0.2,0,True,False,False
4,4.4,3.0,1.3,0.2,0,True,False,False
...,...,...,...,...,...,...,...,...
136,7.9,3.8,6.4,2.0,2,False,False,True
137,7.6,3.0,6.6,2.1,2,False,False,True
138,7.7,3.8,6.7,2.2,2,False,False,True
139,7.7,2.8,6.7,2.0,2,False,False,True


In [129]:
#
# We've removed the irisname column, and with our "" 
# we don't need the irisnum of the columns need to be numeric, we'll drop irisname
ROW = 0
COLUMN = 1
df_model1 = df_tidy_cat.drop('irisnum', axis=COLUMN )
df_model1

,sepallen,sepalwid,petallen,petalwid,is_setosa,is_versicolor,is_virginica
0,4.6,3.6,1.0,0.2,True,False,False
1,4.3,3.0,1.1,0.1,True,False,False
2,5.0,3.2,1.2,0.2,True,False,False
3,5.8,4.0,1.2,0.2,True,False,False
4,4.4,3.0,1.3,0.2,True,False,False
...,...,...,...,...,...,...,...
136,7.9,3.8,6.4,2.0,False,False,True
137,7.6,3.0,6.6,2.1,False,False,True
138,7.7,3.8,6.7,2.2,False,False,True
139,7.7,2.8,6.7,2.0,False,False,True


In [130]:
#
# once we have all the columns we want, let's create an index of their names...

#
# Let's make sure we have all of our helpful variables in one place 
#       To be adapted if we drop/add more columns...
#

#
# let's keep our column names in variables, for reference
#
COLUMNS = df_model1.columns            # "list" of columns
print(f"COLUMNS is {COLUMNS}\n")  
  # It's a "pandas" list, called an Index
  # use it just as a Python list of strings:
print(f"COLUMNS[0] is {COLUMNS[0]}\n")

# let's create a dictionary to look up any column index by name
COL_INDEX = {}
for i, name in enumerate(COLUMNS):
    COL_INDEX[name] = i  # using the name (as key), look up the value (i)
print(f"COL_INDEX is {COL_INDEX}\n\n")


#
# and our "species" names
#

# all of scikit-learn's ML routines need numbers, not strings
#   ... even for categories/classifications (like species!)
#   so, we will convert the flower-species to numbers:

SPECIES = ['setosa','versicolor','virginica']   # int to str
SPECIES_INDEX = {'setosa':0,'versicolor':1,'virginica':2}  # str to int

# Let's try it out...
for name in SPECIES:
    print(f"{name} maps to {SPECIES_INDEX[name]}")

COLUMNS is Index(['sepallen', 'sepalwid', 'petallen', 'petalwid', 'is_setosa',
       'is_versicolor', 'is_virginica'],
      dtype='object')

COLUMNS[0] is sepallen

COL_INDEX is {'sepallen': 0, 'sepalwid': 1, 'petallen': 2, 'petalwid': 3, 'is_setosa': 4, 'is_versicolor': 5, 'is_virginica': 6}


setosa maps to 0
versicolor maps to 1
virginica maps to 2


In [131]:
#
# We _could_ reweight our columns...  DON'T DO THIS!
# For example, if petalwid were "worth" 20x more than the others?
# 

# df_model1['petalwid'] *= 20
# df_model1

#
# But, with NNets, the whole goal of the network is to _adaptively_ weight each feature...
#
#      That's really all the network is doing... !?!!
#      Let's see it in action:
#

In [132]:
#
# let's convert our dataframe to a numpy array, named A
#
A = df_model1.to_numpy()   
A = A.astype('float64')    # many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A[:,:])

# print(A)  # the five rows above is probably enough...   

[[4.6 3.6 1.  0.2 1.  0.  0. ]
 [4.3 3.  1.1 0.1 1.  0.  0. ]
 [5.  3.2 1.2 0.2 1.  0.  0. ]
 [5.8 4.  1.2 0.2 1.  0.  0. ]
 [4.4 3.  1.3 0.2 1.  0.  0. ]
 [4.4 3.2 1.3 0.2 1.  0.  0. ]
 [4.5 2.3 1.3 0.3 1.  0.  0. ]
 [4.7 3.2 1.3 0.2 1.  0.  0. ]
 [5.  3.5 1.3 0.3 1.  0.  0. ]
 [5.4 3.9 1.3 0.4 1.  0.  0. ]
 [5.5 3.5 1.3 0.2 1.  0.  0. ]
 [4.4 2.9 1.4 0.2 1.  0.  0. ]
 [4.6 3.4 1.4 0.3 1.  0.  0. ]
 [4.6 3.2 1.4 0.2 1.  0.  0. ]
 [4.8 3.  1.4 0.1 1.  0.  0. ]
 [4.8 3.  1.4 0.3 1.  0.  0. ]
 [4.9 3.  1.4 0.2 1.  0.  0. ]
 [5.  3.6 1.4 0.2 1.  0.  0. ]
 [5.  3.3 1.4 0.2 1.  0.  0. ]
 [5.1 3.5 1.4 0.2 1.  0.  0. ]
 [5.1 3.5 1.4 0.3 1.  0.  0. ]
 [5.2 3.4 1.4 0.2 1.  0.  0. ]
 [5.5 4.2 1.4 0.2 1.  0.  0. ]
 [4.6 3.1 1.5 0.2 1.  0.  0. ]
 [4.9 3.1 1.5 0.1 1.  0.  0. ]
 [4.9 3.1 1.5 0.1 1.  0.  0. ]
 [4.9 3.1 1.5 0.1 1.  0.  0. ]
 [5.  3.4 1.5 0.2 1.  0.  0. ]
 [5.1 3.8 1.5 0.3 1.  0.  0. ]
 [5.1 3.7 1.5 0.4 1.  0.  0. ]
 [5.1 3.4 1.5 0.2 1.  0.  0. ]
 [5.2 3.5 1.5 0.2 1.  0.  0. ]
 [5.3 3.

In [133]:
#
# nice to have NUM_ROWS and NUM_COLS around
#
NUM_ROWS, NUM_COLS = A.shape
print(f"\nThe dataset has {NUM_ROWS} rows and {NUM_COLS} cols")


The dataset has 141 rows and 7 cols


### We have full access to individual data elements

Checking on our observations and attributes

In [134]:
# let's use all of our variables, to reinforce that we have
# (1) names...
# (2) access and control...

# choose a row index, n:
n = 42
print(f"flower #{n} is {A[n]}")
species = ""

for i in range(len(COLUMNS)):
    colname = COLUMNS[i]
    value = A[n][i]
    print(f"  Its {colname} is {value}")
    if "setosa" in colname: species = "setosa"
    elif "versicolor" in colname: species = "versicolor"
    elif "virginica" in colname: species = "virginica"

print()
print(f"  and its species name: {species}")

flower #42 is [5.4 3.9 1.7 0.4 1.  0.  0. ]
  Its sepallen is 5.4
  Its sepalwid is 3.9
  Its petallen is 1.7
  Its petalwid is 0.4
  Its is_setosa is 1.0
  Its is_versicolor is 0.0
  Its is_virginica is 0.0

  and its species name: virginica


### Defining X_all (our features) and y_all (our target to predict) 

In [135]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:,0:4]   # X (features) ... is all rows, columns 0, 1, 2, 3
y_all = A[:,4:]    # y (labels) ... is all rows, columns 4, 5, and 6

print(f"y_all (just the labels/species, first few rows) are \n {y_all[0:5]}")
print()
print(f"X_all (just the features, first few rows) are \n {X_all[0:5]}")

+++ Start of data definitions +++

y_all (just the labels/species, first few rows) are 
 [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]

X_all (just the features, first few rows) are 
 [[4.6 3.6 1.  0.2]
 [4.3 3.  1.1 0.1]
 [5.  3.2 1.2 0.2]
 [5.8 4.  1.2 0.2]
 [4.4 3.  1.3 0.2]]


### Permute...

In [136]:
#
# we can scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
print(f"The scrambled labels/species are \n {y_all[0:5]}")
print()
print(f"The corresponding data rows are \n {X_all[0:5]}")

The scrambled labels/species are 
 [[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]

The corresponding data rows are 
 [[4.9 3.  1.4 0.2]
 [7.6 3.  6.6 2.1]
 [7.2 3.  5.8 1.6]
 [6.1 2.8 4.7 1.2]
 [6.7 3.  5.2 2.3]]


### Separate into training and testing
+ X_train, y_train will be used for model-building
+ X_test, y_test will be used for testing, later

In [137]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"+++ Testing +++   Held-out data... (testing data: {len(y_test)})\n")
print(f"y_test: {y_test[0:5,:]}\n")
print(f"X_test (few rows): {X_test[0:5,:]}")  # 5 rows
print("\n")
print(f"+++ Training +++   Data used for modeling... (training data: {len(y_train)})\n")
print(f"y_train: {y_train[0:5,:]}\n")
print(f"X_train (few rows): {X_train[0:5,:]}")  # 5 rows

training with 112 rows;  testing with 29 rows

+++ Testing +++   Held-out data... (testing data: 29)

y_test: [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

X_test (few rows): [[6.2 2.2 4.5 1.5]
 [6.  2.7 5.1 1.6]
 [6.8 2.8 4.8 1.4]
 [6.3 3.3 4.7 1.6]
 [7.6 3.  6.6 2.1]]


+++ Training +++   Data used for modeling... (training data: 112)

y_train: [[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]

X_train (few rows): [[4.8 3.  1.4 0.3]
 [6.  2.2 5.  1.5]
 [5.5 2.6 4.4 1.2]
 [5.  3.4 1.5 0.2]
 [7.2 3.2 6.  1.8]]


### NNets often need a ***Scaler***

This scales all of the data so that it's of a similar size, e.g., -1 to 1

Notice that this cell produces
+ X_train_scaled &nbsp; (the scaled training data)
   + y_train_scaled is the same as y_train &nbsp; (no scaling needed for the target)
+ X_test_scaled &nbsp; (the scaled testing data)
   + y_test_scaled is the same as y_test &nbsp; (no scaling needed for the target)

After this, we just use the scaled data...

In [138]:
#
# for NNets, it's important to keep the feature values near 0, say -1. to 1. or so
#    This is done through the "StandardScaler" in scikit-learn
# 
from sklearn.preprocessing import StandardScaler

#
# do we want to use a Scaler?
#
USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    scaler = StandardScaler()
    scaler.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scaler = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scaler.fit(X_train)  # still need to fit, though it does not change...

scaler   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Here are our scaled training and testing sets:

X_train_scaled = scaler.transform(X_train) # scale!
X_test_scaled = scaler.transform(X_test) # scale!

y_train_scaled = y_train.copy()  # the predicted/desired labels are not scaled
y_test_scaled = y_test.copy()  # not using the scaler



### Intuition-building for the Scaled data

In [139]:
#
# let's create a table for showing our data and its predictions...
#
def ascii_table(X,y,scaler_to_invert=None):
    """ print a table of inputs and outputs """
    np.set_printoptions(precision=2)  # Let's use less precision
    if scaler_to_invert == None:  # don't use the scaler
        X = X
    else:
        X = scaler_to_invert.inverse_transform(X)
    print(f"{'input ':>58s} -> {'pred':^7s} {'des':<5s}") 
    for i in range(len(y)):
        # whoa! serious f-string formatting:
        print(f"{str(X[i,0:4]):>58s} -> {'?':^7s} {str(y[i]):<21s}")   # !s is str ...
    print()
    
# to show the table with the scaled data:
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],None)

# to show the table with the original data:
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],scaler_to_invert=scaler)

                                                    input  ->  pred   des  
                                 [-1.24 -0.13 -1.33 -1.18] ->    ?    [1. 0. 0.]           
                                 [ 0.19 -1.92  0.71  0.42] ->    ?    [0. 0. 1.]           
                                 [-0.41 -1.02  0.37  0.02] ->    ?    [0. 1. 0.]           
                                 [-1.    0.76 -1.27 -1.32] ->    ?    [1. 0. 0.]           
                                     [1.62 0.32 1.28 0.82] ->    ?    [0. 0. 1.]           

                                                    input  ->  pred   des  
                                         [4.8 3.  1.4 0.3] ->    ?    [1. 0. 0.]           
                                         [6.  2.2 5.  1.5] ->    ?    [0. 0. 1.]           
                                         [5.5 2.6 4.4 1.2] ->    ?    [0. 1. 0.]           
                                         [5.  3.4 1.5 0.2] ->    ?    [1. 0. 0.]           
                   

### Training ML models is no problem

that is, if the library does it for us! &nbsp;&nbsp; 😃 🧶 ⛄️ 🌊 🪺 🦔 :-)

In [140]:
from sklearn.neural_network import MLPClassifier

#
# Here's where you can change the number of hidden layers
# and number of neurons!  It's in the tuple  hidden_layer_sizes:
#
nn_classifier = MLPClassifier(hidden_layer_sizes=(6,7),  
                    # hidden_layer_sizes=(6,7)   means   4 inputs -> 6 hidden -> 7 hidden -> 3 outputs
                    max_iter=500,      # how many times to train
                    # activation="tanh", # the "activation function" input -> output
                    # solver='sgd',      # the algorithm for optimizing weights
                    verbose=True,      # False to "mute" the training
                    shuffle=True,      # reshuffle the training epochs?
                    random_state=None, # set for reproduceability
                    learning_rate_init=.1,       # learning rate: the amt of error to backpropagate!
                    learning_rate = 'adaptive')  # soften feedback as it converges

# documentation:
# scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html 
#     Try other network sizes / other parameters ...

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_classifier.fit(X_train_scaled, y_train_scaled)
print("\n++++++++++  TRAINING:   end  +++++++++++++++")
print(f"The analog prediction error (the loss) is {nn_classifier.loss_}")



++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 2.27041274
Iteration 2, loss = 1.92073327
Iteration 3, loss = 1.80970034
Iteration 4, loss = 1.70841495
Iteration 5, loss = 1.49959573
Iteration 6, loss = 1.21454963
Iteration 7, loss = 0.91652817
Iteration 8, loss = 0.72200035
Iteration 9, loss = 0.67981873
Iteration 10, loss = 0.67070645
Iteration 11, loss = 0.60001402
Iteration 12, loss = 0.49955865
Iteration 13, loss = 0.41958199
Iteration 14, loss = 0.38064682
Iteration 15, loss = 0.35349545
Iteration 16, loss = 0.32620388
Iteration 17, loss = 0.29492476
Iteration 18, loss = 0.25831591
Iteration 19, loss = 0.21136983
Iteration 20, loss = 0.16183592
Iteration 21, loss = 0.12729065
Iteration 22, loss = 0.11460348
Iteration 23, loss = 0.11112534
Iteration 24, loss = 0.10346977
Iteration 25, loss = 0.10110075
Iteration 26, loss = 0.10056551
Iteration 27, loss = 0.09549214
Iteration 28, loss = 0.09249417
Iteration 29, loss = 0.09124377
Iteration 30, loss = 0.08488683

### Now, let's test!

In [141]:
#
# how did it do on the testing data?
#

def get_species(A):
    """ returns the species for A ~ [1 0 0] or [0 1 0] or ... """
    for i in range(len(SPECIES)):
        if A[i] == 1: 
            return SPECIES[i]  # note that this "takes the first one"
    return "no species" 

SEE_PROBS = False

def ascii_table_for_classifier(Xsc,y,nn,scaler):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(Xsc)            # all predictions
    prediction_probs = nn.predict_proba(Xsc) # all prediction probabilities
    Xpr = scaler.inverse_transform(Xsc)      # Xpr is the "X to print": the unscaled data
    # count correct
    num_correct = 0
    # printing
    print(f"{'input ':>28s} -> {'pred':^12s} {'des.':^12s}") 
    for i in range(len(y)):
        pred = predictions[i]
        pred_probs = str(prediction_probs[i,:])
        desired = y[i].astype(int)
        # print(pred, desired, pred_probs)
        pred_species = get_species(pred)
        des_species  = get_species(desired)
        if pred_species != des_species: result = "  incorrect: " + pred_probs
        else: result = "  correct" + (": "+pred_probs if SEE_PROBS else "") ; num_correct += 1
        # Xpr = Xsc  # if you want to see the scaled versions
        print(f"{Xpr[i,0:4]!s:>28s} -> {pred_species:^12s} {des_species:12s} {result:^10s}") 
    print(f"\ncorrect predictions: {num_correct} out of {len(y)}")
    
#
# let's see how it did on the test data (also the training data!)
#
ascii_table_for_classifier(X_test_scaled,
                           y_test_scaled,
                           nn_classifier,
                           scaler)   


                      input  ->     pred         des.    
           [6.2 2.2 4.5 1.5] ->  versicolor  versicolor     correct 
           [6.  2.7 5.1 1.6] ->  virginica   versicolor     incorrect: [8.69e-09 2.11e-08 1.00e+00]
           [6.8 2.8 4.8 1.4] ->  versicolor  versicolor     correct 
           [6.3 3.3 4.7 1.6] ->  versicolor  versicolor     correct 
           [7.6 3.  6.6 2.1] ->  virginica   virginica      correct 
           [6.2 3.4 5.4 2.3] ->  virginica   virginica      correct 
           [6.5 3.  5.2 2. ] ->  virginica   virginica      correct 
           [6.4 3.2 5.3 2.3] ->  virginica   virginica      correct 
           [5.1 3.7 1.5 0.4] ->    setosa    setosa         correct 
           [5.5 2.4 3.8 1.1] ->  versicolor  versicolor     correct 
           [4.9 3.1 1.5 0.1] ->    setosa    setosa         correct 
           [5.5 2.4 3.7 1. ] ->  versicolor  versicolor     correct 
           [5.6 3.  4.5 1.5] ->  versicolor  versicolor     correct 
           [7.

### Let's see the network's weights...

In [142]:
#
# We don't usually look inside the NNet, but we can: it's open-box modeling...
#
if True:  # do we want to see all of the parameters?
    np.set_printoptions(precision=2)  # Let's use less precision
    nn = nn_classifier  # less to type?
    print("\n\n+++++ parameters, weights, etc. +++++\n")
    print(f"\nweights/coefficients:\n")
    for i, wts in enumerate(nn.coefs_):
        print(f"[[ Layer {i} ]]\n   has shape = {wts.shape} and weights =\n{wts}")
        print(f"   with intercepts:\n {nn.intercepts_[i]}\n")
    print()
    print(f"\nall parameters: {nn.get_params()}")



+++++ parameters, weights, etc. +++++


weights/coefficients:

[[ Layer 0 ]]
   has shape = (4, 6) and weights =
[[-0.87 -0.78 -0.11 -0.18 -1.19 -0.52]
 [ 0.58 -0.4  -1.5  -0.44  0.68  0.45]
 [-1.3   2.39  0.37  3.43 -0.01 -1.84]
 [-1.21  1.78  1.25  1.08 -0.82 -1.85]]
   with intercepts:
 [-0.37 -2.17  1.26 -2.47 -1.08  1.24]

[[ Layer 1 ]]
   has shape = (6, 7) and weights =
[[-3.82e-01 -9.41e-01  1.87e+00 -8.81e-01 -1.56e+00  1.61e+00  4.26e-01]
 [-1.46e-05 -7.30e-01 -1.56e+00  1.39e+00 -3.18e+00 -1.18e+00  2.00e+00]
 [-1.51e-04  1.54e+00 -1.53e+00  9.72e-01  1.05e+00 -1.80e+00  1.43e-01]
 [-2.38e-01 -2.90e+00 -1.18e+00  1.82e+00 -4.53e+00 -3.07e-01  4.64e+00]
 [-1.63e-01  1.06e+00  6.45e-01  2.92e-01 -4.30e-02  1.69e-01  1.99e-01]
 [ 1.15e-04  1.28e-01  2.18e+00 -9.34e-01  6.12e-01  1.09e+00 -7.19e-01]]
   with intercepts:
 [-1.13  0.06  0.64 -0.08  2.79 -0.4  -1.35]

[[ Layer 2 ]]
   has shape = (7, 3) and weights =
[[-0.8   0.08  0.54]
 [-1.67  0.55 -0.23]
 [ 1.77 -1.07 -1.87]


### The predictive model

In [143]:
#
# final predictive model (random forests), with tuned parameters + ALL data incorporated
#

def get_species(A):
    """ returns the species for A ~ [1 0 0] or [0 1 0] or ... """
    for i in range(len(SPECIES)):
        if A[i] == 1: 
            return SPECIES[i]  # note that this "takes the first one"
    return "no species" 


def predictive_model( Features, MODEL, SCALER ):
    """ input: a list of four features 
                [ sepallen, sepalwid, petallen, petalwid ]
        output: the predicted species of iris, from
                  setosa (0), versicolor (1), virginica (2)
    """
    our_features = np.asarray([Features])                 # extra brackets needed
    scaled_features = SCALER.transform(our_features)      # we have to scale the features into "scaled space"
    predicted_cat = MODEL.predict(scaled_features)        # then, the nnet can predict our "cat" variables
    prediction_probs = nn.predict_proba(scaled_features) # all prediction probabilities
    # our_features = SCALER.inverse_transform(scaled_features)  # we can convert back (optional!)
    predicted_species = get_species(predicted_cat[0])     # (it's extra-nested) get the species name
    return predicted_species, prediction_probs
   
#
# Try it!
# 
# Features = eval(input("Enter new Features: "))
#
Features = [6.7,3.3,5.7,0.1]  # [5.8,2.7,4.1,1.0] [4.6,3.6,3.0,2.2] [6.7,3.3,5.7,2.1]

LoF = [
[4.8, 3.1, 1.6, 0.2 ],   # actually setosa
[5.7, 2.9, 4.2, 1.3 ],   # actually versicolor
[5.8, 2.7, 5.1, 1.9 ],   # actually virginica
[5.2, 4.1, 1.5, 0.1 ],   # actually setosa
[5.4, 3.4, 1.5, 0.4 ],   # actually setosa
[5.1, 2.5, 3.0, 1.1 ],   # actually versicolor
[6.2, 2.9, 4.3, 1.3 ],   # actually versicolor
[6.3, 3.3, 6.0, 2.5 ],   # actually virginica
[5.7, 2.8, 4.1, 1.3 ],   # actually virginica  <-- almost always wrong!
]

SEE_PROBS = False

# run on each one:
for Features in LoF:
    MODEL = nn_classifier
    SCALER = scaler
    name, probs = predictive_model( Features, MODEL, SCALER )  # pass in the model, too!
    prob_str = "   with probs: " + str(probs) if SEE_PROBS == True else ""
    print(f"I predict {name:>12s} from the features {Features}  {prob_str}")    # Answers in the assignment...

I predict       setosa from the features [4.8, 3.1, 1.6, 0.2]  
I predict   versicolor from the features [5.7, 2.9, 4.2, 1.3]  
I predict    virginica from the features [5.8, 2.7, 5.1, 1.9]  
I predict       setosa from the features [5.2, 4.1, 1.5, 0.1]  
I predict       setosa from the features [5.4, 3.4, 1.5, 0.4]  
I predict   versicolor from the features [5.1, 2.5, 3.0, 1.1]  
I predict   versicolor from the features [6.2, 2.9, 4.3, 1.3]  
I predict    virginica from the features [6.3, 3.3, 6.0, 2.5]  
I predict   versicolor from the features [5.7, 2.8, 4.1, 1.3]  


<br>

#### We _could_ use cross-validation to find the "right"/"best" size and shape of the NNet...

<font color="Coral">But, let's not...</font>



### <font color="DodgerBlue">Instead, we'll explore two new directions:</font> 
+ regression and 
+ feature-predictability

#### The next example...
+ is one in which we estimate a botanical feature ... 
+ ... from the other four columns -- including the species!

In [144]:
#
# What shall we predict today?
print(COL_INDEX)
print()
print(COLUMNS)

# Let's first predict sepal length ('sepallen', column index 0)

{'sepallen': 0, 'sepalwid': 1, 'petallen': 2, 'petalwid': 3, 'is_setosa': 4, 'is_versicolor': 5, 'is_virginica': 6}

Index(['sepallen', 'sepalwid', 'petallen', 'petalwid', 'is_setosa',
       'is_versicolor', 'is_virginica'],
      dtype='object')


In [145]:
#
# Here we set up for a regression model that will predict 'sepallen'  (column index 0)
#
#   sepal length  'sepallen' (column index 0)  <-- our target for _regression_

# We will use
#
#   sepal width   'sepalwid' (column index 1)
#   petal length  'petallen' (column index 2)
#   petal width   'petalwid' (column index 3)
#   _and_
#   species       'irisnum'  (column index 4)    # No problem - we can use this as a feature!

print("+++ Start of data-assembly for feature-regression! +++\n")
# construct the correct X_all from the columns we want
# we use np.concatenate to combine parts of the dataset to get all-except-column 0:
#                     exclude 0  , include 1 to the end
X_all = np.concatenate( (A[:,0:0], A[:,1:]), axis=1)    # includes columns 1, 2, 3, and 4

# if we wanted all-except-column 1:   X_all = np.concatenate( (A[:,0:1], A[:,2:]),axis=1)  # columns 0, 2, 3, and 4
# if we wanted all-except-column 2:   X_all = np.concatenate( (A[:,0:2], A[:,3:]),axis=1)  # columns 0, 1, 3, and 4
# if we wanted all-except-column 3:   X_all = np.concatenate( (A[:,0:3], A[:,4:]),axis=1)  # columns 0, 1, 2, and 4
# if we wanted all-except-column 4:   X_all = np.concatenate( (A[:,0:4], A[:,5:]),axis=1)  # columns 0, 1, 2, and 3
# (slicing is forgiving)


y_all = A[:,0]                    # y (labels) ... is all of column 0, sepallen (sepal length) 
#                                 # change the line above to make other columns the target (y_all)
print(f"y_all is \n {y_all}")
print() 
print(f"X_all (the features, first few rows) is \n {X_all[:5,:]}")


+++ Start of data-assembly for feature-regression! +++

y_all is 
 [4.6 4.3 5.  5.8 4.4 4.4 4.5 4.7 5.  5.4 5.5 4.4 4.6 4.6 4.8 4.8 4.9 5.
 5.  5.1 5.1 5.2 5.5 4.6 4.9 4.9 4.9 5.  5.1 5.1 5.1 5.2 5.3 5.4 5.7 4.7
 4.8 5.  5.  5.  5.1 5.1 5.4 5.4 5.7 4.8 5.1 7.  4.9 5.  5.  5.7 5.6 5.5
 5.5 5.2 5.6 5.8 5.5 5.5 5.8 6.  6.1 5.6 5.8 5.6 5.7 5.9 6.4 5.5 6.3 6.6
 6.7 5.4 5.6 5.7 6.  6.  6.2 6.4 6.1 6.5 6.6 6.1 6.1 6.3 6.7 5.9 6.8 6.3
 6.9 6.7 6.  4.9 6.  6.2 5.6 6.1 6.3 5.7 6.  6.3 5.8 5.8 5.9 6.3 6.5 6.9
 6.5 6.7 6.4 6.4 6.2 6.9 6.4 6.5 6.8 6.1 6.3 6.3 6.4 6.4 6.7 6.7 6.7 6.9
 6.5 6.7 7.2 6.8 7.1 7.2 7.2 7.4 7.7 7.3 7.9 7.6 7.7 7.7 7.7]

X_all (the features, first few rows) is 
 [[3.6 1.  0.2 1.  0.  0. ]
 [3.  1.1 0.1 1.  0.  0. ]
 [3.2 1.2 0.2 1.  0.  0. ]
 [4.  1.2 0.2 1.  0.  0. ]
 [3.  1.3 0.2 1.  0.  0. ]]


### Permute!

<font size="-2">I like this because it adds an element of randomness to the whole process...</font>

In [146]:
#
# we scramble the data, to give a different TRAIN/TEST split each time...
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
print("target values to predict: \n",y_all)
print("\nfeatures (a few)\n", X_all[0:5,:])

target values to predict: 
 [5.6 6.5 6.  6.8 6.7 4.9 6.3 6.7 7.1 6.9 7.  5.6 5.  7.2 5.7 5.  6.7 4.8
 4.6 6.5 5.1 6.4 4.5 5.4 6.4 4.4 5.9 7.7 5.4 5.  5.5 6.7 7.2 4.4 6.  6.1
 6.4 5.1 6.2 5.1 6.7 6.  6.1 7.7 4.9 5.1 5.7 5.7 5.6 6.9 5.  6.5 6.7 6.7
 5.5 5.4 5.2 5.  7.3 4.3 5.7 5.8 4.8 5.5 5.9 6.  4.4 5.1 4.6 5.5 7.7 6.5
 4.9 6.  7.6 5.  6.1 6.9 4.8 5.4 5.5 6.7 5.2 4.8 5.7 4.9 6.3 6.3 5.1 7.7
 5.8 5.  6.9 5.2 5.8 5.8 6.3 6.6 7.2 6.3 4.6 6.8 5.5 5.7 6.  5.6 6.8 5.8
 6.5 5.6 6.6 5.5 6.1 4.7 6.1 6.4 6.2 5.4 4.9 5.  7.4 6.2 5.1 5.1 5.  4.7
 4.6 6.1 7.9 5.9 6.3 5.  6.3 6.4 5.8 5.3 6.4 6.4 5.6 4.9 6.3]

features (a few)
 [[3.  4.1 1.3 0.  1.  0. ]
 [3.2 5.1 2.  0.  0.  1. ]
 [3.4 4.5 1.6 0.  1.  0. ]
 [3.  5.5 2.1 0.  0.  1. ]
 [2.5 5.8 1.8 0.  0.  1. ]]


### Split into training and testing sets

We'll stick with 80% training, 20% testing...

In [147]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)})")
print(f"y_test: {y_test}\n")
print(f"X_test (few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)})")
print(f"y_train: {y_train}\n")
print(f"X_train (few rows): {X_train[0:5,:]}")  # 5 rows

training with 112 rows;  testing with 29 rows

Held-out data... (testing data: 29)
y_test: [6.5 6.5 5.5 5.1 5.4 5.5 5.  5.6 6.5 6.5 6.2 6.1 5.8 5.7 6.  5.9 5.7 5.1
 5.  7.7 6.3 5.1 5.1 5.7 5.8 5.4 5.4 6.8 5. ]

X_test (few rows): [[3.  5.5 1.8 0.  0.  1. ]
 [2.8 4.6 1.5 0.  1.  0. ]
 [2.4 3.8 1.1 0.  1.  0. ]
 [3.7 1.5 0.4 1.  0.  0. ]
 [3.4 1.7 0.2 1.  0.  0. ]]

Data used for modeling... (training data: 112)
y_train: [6.8 5.9 4.7 4.7 6.7 6.  5.5 4.8 4.4 5.  6.1 6.7 4.9 5.6 5.5 6.  5.  7.4
 6.3 5.  4.9 5.1 6.7 4.9 6.  4.6 4.6 6.1 7.2 7.2 4.9 5.2 6.3 5.8 6.3 6.3
 7.1 7.7 5.2 6.7 7.9 5.6 7.7 5.7 6.4 4.8 4.4 5.5 5.4 6.7 4.5 4.6 6.7 4.4
 6.9 6.9 6.6 4.8 5.3 6.6 5.  6.4 5.7 5.8 6.5 4.8 4.9 6.9 7.3 6.1 4.6 4.3
 5.  6.9 5.  5.5 5.1 7.  5.9 5.5 6.4 6.8 5.6 6.4 6.2 7.2 6.2 5.  5.1 5.6
 6.  5.2 6.1 6.3 5.8 5.1 5.6 7.6 6.3 6.4 6.4 6.1 5.7 5.4 5.8 6.  6.7 6.4
 6.7 4.9 7.7 6.3]

X_train (few rows): [[3.  5.5 2.1 0.  0.  1. ]
 [3.2 4.8 1.8 0.  1.  0. ]
 [3.2 1.6 0.2 1.  0.  0. ]
 [3.2 1.3 0.2 1.  0

### It's a nnet - this time, we _should_ use a <u>Scaler</u>

<font size="-2">Note that, for our <u>own</u> neural nets, this is performed by our senses:
+ our sense of hearing scales sounds from whispers to shouts... 
    + ... into a single amplitude for interpretation. 
    + and, we keep the scale itself as an additional signal!
+ similarly, our visual system scales across many orders of magnitude of brightness
    + plus, our eyes can "adapt" to low light (and, less, to bright-light )
+ less dramatically, for taste, touch, and smell...
+ this scaling 

In [148]:
#
# for NNets, it's important to keep the feature values near 0, say -1. to 1. or so
#    This is done through the "StandardScaler" in scikit-learn
#
from sklearn.preprocessing import StandardScaler

USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    scaler = StandardScaler()
    scaler.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scaler = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scaler.fit(X_train)  # still need to fit, though it does not change...

scaler   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Here are our scaled training and testing sets:

X_train_scaled = scaler.transform(X_train) # scale!
X_test_scaled = scaler.transform(X_test) # scale!

y_train_scaled = y_train  # the predicted/desired labels are not scaled
y_test_scaled = y_test  # not using the scaler

# reused from above - seeing the scaled data 
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],None)

# reused from above - seeing the unscaled data (inverting the scaler)
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],scaler)

                                                    input  ->  pred   des  
                                 [-0.04  0.92  1.12 -0.66] ->    ?    6.8                  
                                 [ 0.46  0.52  0.73 -0.66] ->    ?    5.9                  
                                 [ 0.46 -1.31 -1.38  1.51] ->    ?    4.7                  
                                 [ 0.46 -1.49 -1.38  1.51] ->    ?    4.7                  
                                 [ 0.71  1.03  1.65 -0.66] ->    ?    6.7                  

                                                    input  ->  pred   des  
                                         [3.  5.5 2.1 0. ] ->    ?    6.8                  
                                         [3.2 4.8 1.8 0. ] ->    ?    5.9                  
                                         [3.2 1.6 0.2 1. ] ->    ?    4.7                  
                                         [3.2 1.3 0.2 1. ] ->    ?    4.7                  
                   

### Create and train the model

Note that hidden_layer_sizes is an important parameter.
+ for this hw, we're simply trying things out
+ no need to optimize via cross-validation

In [149]:
#
# MLPRegressor predicts _floating-point_ outputs
#

from sklearn.neural_network import MLPRegressor

nn_regressor = MLPRegressor(hidden_layer_sizes=(6,7), 
                    max_iter=500,          # how many training epochs
                    verbose=True,          # do we want to watch as it trains?
                    shuffle=True,          # shuffle each epoch?
                    random_state=None,     # use for reproducibility
                    learning_rate_init=.1, # how much of each error to back-propagate
                    learning_rate = 'adaptive')  # how to handle the learning_rate

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_regressor.fit(X_train_scaled, y_train_scaled)
print("++++++++++  TRAINING:   end  +++++++++++++++")
print(f"The (squared) prediction error (the loss) is {nn_regressor.loss_:<6.3f}")
print(f"And, its square root:         {nn_regressor.loss_ ** 0.5:<6.3f}")
print()



++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 22.79464842
Iteration 2, loss = 18.96879149
Iteration 3, loss = 16.14703887
Iteration 4, loss = 13.28203054
Iteration 5, loss = 10.03898869
Iteration 6, loss = 7.55733514
Iteration 7, loss = 6.55390828
Iteration 8, loss = 4.87306748
Iteration 9, loss = 2.59979089
Iteration 10, loss = 0.90642904
Iteration 11, loss = 0.61300580
Iteration 12, loss = 1.37192737
Iteration 13, loss = 2.00269514
Iteration 14, loss = 1.93495132
Iteration 15, loss = 1.40554815
Iteration 16, loss = 0.84212809
Iteration 17, loss = 0.45585135
Iteration 18, loss = 0.25045210
Iteration 19, loss = 0.19168506
Iteration 20, loss = 0.23534458
Iteration 21, loss = 0.31700784
Iteration 22, loss = 0.36314619
Iteration 23, loss = 0.35154826
Iteration 24, loss = 0.29795980
Iteration 25, loss = 0.23005333
Iteration 26, loss = 0.18425258
Iteration 27, loss = 0.17633910
Iteration 28, loss = 0.19096352
Iteration 29, loss = 0.20061894
Iteration 30, loss = 0.196

In [150]:
#
# Note that square-root of the mean-squared-error is an "expected error" in predicting our feature
#
# It's not the only estimate of expected error, but it's easier to think about than the squared error (which has the wrong units)
#

### Let's test!  

Testing is different for regression!  
<font size="-1">
+ it's floating-point, so it's no longer "right" or "wrong"
+ instead, the _difference_ is what matters
+ in fact, the <u> _absolute difference_ </u> is usually what matters: </font>

In [151]:
#
# how did it do? Try out the TEST data...
#

def ascii_table_for_regressor(Xsc,y,nn,scaler):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(Xsc) # all predictions
    Xpr = scaler.inverse_transform(Xsc)  # Xpr is the "X to print": unscaled data!
    # measure error
    error = 0.0
    # printing
    print(f"{'input ':>35s} ->  {'pred':^6s}  {'des.':^6s}   {'absdiff':^10s}") 
    for i in range(len(y)):
        pred = predictions[i]
        desired = y[i]
        result = abs(desired - pred)
        error += result
        # Xpr = Xsc   # if you'd like to see the scaled values
        print(f"{Xpr[i,:]!s:>35s} ->  {pred:<+6.2f}  {desired:<+6.2f}   {result:^10.2f}") 

    print("\n" + "+++++   +++++   +++++           ")
    print(f"average abs diff error:   {error/len(y):<6.3f}")
    print("+++++   +++++   +++++           ")
    
#
# let's see how it did on the test data (also the training data!)
#
ascii_table_for_regressor(X_test_scaled,
                          y_test_scaled,
                          nn_regressor,
                          scaler)   # this is our own f'n, above


                             input  ->   pred    des.     absdiff  
          [3.  5.5 1.8 0.  0.  1. ] ->  +6.66   +6.50       0.16   
          [2.8 4.6 1.5 0.  1.  0. ] ->  +6.10   +6.50       0.40   
          [2.4 3.8 1.1 0.  1.  0. ] ->  +5.51   +5.50       0.01   
          [3.7 1.5 0.4 1.  0.  0. ] ->  +5.05   +5.10       0.05   
          [3.4 1.7 0.2 1.  0.  0. ] ->  +4.94   +5.40       0.46   
          [3.5 1.3 0.2 1.  0.  0. ] ->  +5.06   +5.50       0.44   
          [3.4 1.5 0.2 1.  0.  0. ] ->  +4.96   +5.00       0.04   
          [2.9 3.6 1.3 0.  1.  0. ] ->  +5.35   +5.60       0.25   
          [3.  5.2 2.  0.  0.  1. ] ->  +6.33   +6.50       0.17   
          [3.2 5.1 2.  0.  0.  1. ] ->  +6.34   +6.50       0.16   
          [2.8 4.8 1.8 0.  0.  1. ] ->  +5.95   +6.20       0.25   
          [3.  4.6 1.4 0.  1.  0. ] ->  +6.34   +6.10       0.24   
          [4.  1.2 0.2 1.  0.  0. ] ->  +5.43   +5.80       0.37   
          [3.  4.2 1.2 0.  1.  0. ] ->  +6.16   

### Your task:  &nbsp;&nbsp; model ***all*** of the iris features!

Which of the four floating-point iris features is
+ _most_ predictable? &nbsp; (smallest average abs diff error)
+ _least_ predictable? &nbsp; (largest average abs diff error)

<br>

As with the _feature importances_ , it's often the <u>insight</u> we value, more than creating the predictor itself...

In [162]:
"""
~~~~~~~~~~~~
Sepal width
~~~~~~~~~~~~

Here we set up for a regression model that will predict sepal width 'sepalwid' (column index 1)

"""
print("SEPALWID")
print("+++ Start of data-assembly for feature-regression! +++\n")
# construct the correct X_all from the columns we want
X_all = np.concatenate( (A[:,0:1], A[:,2:]),axis=1)  # includes columns 0, 2, 3, and 4

y_all = A[:,1]                    # y (labels) ... is all of column 1, sepalwid

# print(f"y_all is \n {y_all}")
# print() 
# print(f"X_all (the features, first few rows) is \n {X_all[:5,:]}")
# print() 
# print() 



# Scramble the data, to give a different TRAIN/TEST split each time...
print("+++ Scramble the data... +++\n")
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
# print("target values to predict: \n",y_all)
# print("\nfeatures (a few)\n", X_all[0:5,:])



# We next separate into test data and training data ... 
print("+++ 80/20 train-test split... +++\n")
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

# print(f"Held-out data... (testing data: {len(y_test)})")
# print(f"y_test: {y_test}\n")
# print(f"X_test (few rows): {X_test[0:5,:]}")  # 5 rows
# print()
# print(f"Data used for modeling... (training data: {len(y_train)})")
# print(f"y_train: {y_train}\n")
# print(f"X_train (few rows): {X_train[0:5,:]}")  # 5 rows



# For NNets, it's important to keep the feature values near 0, say -1. to 1. or so
#    his is done through the "StandardScaler" in scikit-learn
print("+++ Scale values... +++\n")
from sklearn.preprocessing import StandardScaler

USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    scaler = StandardScaler()
    scaler.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scaler = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scaler.fit(X_train)  # still need to fit, though it does not change...

scaler   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Here are our scaled training and testing sets:

X_train_scaled = scaler.transform(X_train) # scale!
X_test_scaled = scaler.transform(X_test) # scale!

y_train_scaled = y_train  # the predicted/desired labels are not scaled
y_test_scaled = y_test  # not using the scaler

# reused from above - seeing the scaled data 
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],None)

# reused from above - seeing the unscaled data (inverting the scaler)
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],scaler)

# Train
# MLPRegressor predicts _floating-point_ outputs
from sklearn.neural_network import MLPRegressor

nn_regressor = MLPRegressor(hidden_layer_sizes=(6,7), 
                    max_iter=500,          # how many training epochs
                    verbose=True,          # do we want to watch as it trains?
                    shuffle=True,          # shuffle each epoch?
                    random_state=None,     # use for reproducibility
                    learning_rate_init=.1, # how much of each error to back-propagate
                    learning_rate = 'adaptive')  # how to handle the learning_rate

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_regressor.fit(X_train_scaled, y_train_scaled)
print("++++++++++  TRAINING:   end  +++++++++++++++")
print(f"The (squared) prediction error (the loss) is {nn_regressor.loss_:<6.3f}")
print(f"And, its square root:         {nn_regressor.loss_ ** 0.5:<6.3f}")
print()

#
# how did it do? Try out the TEST data...
#
print("SEPALWID TESTING RESULTS")
def ascii_table_for_regressor(Xsc,y,nn,scaler):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(Xsc) # all predictions
    Xpr = scaler.inverse_transform(Xsc)  # Xpr is the "X to print": unscaled data!
    # measure error
    error = 0.0
    # printing
    print(f"{'input ':>35s} ->  {'pred':^6s}  {'des.':^6s}   {'absdiff':^10s}") 
    for i in range(len(y)):
        pred = predictions[i]
        desired = y[i]
        result = abs(desired - pred)
        error += result
        # Xpr = Xsc   # if you'd like to see the scaled values
        print(f"{Xpr[i,:]!s:>35s} ->  {pred:<+6.2f}  {desired:<+6.2f}   {result:^10.2f}") 

    print("\n" + "+++++   +++++   +++++           ")
    print(f"average abs diff error:   {error/len(y):<6.3f}")
    print("+++++   +++++   +++++           ")
    
#
# let's see how it did on the test data (also the training data!)
#
ascii_table_for_regressor(X_test_scaled,
                          y_test_scaled,
                          nn_regressor,
                          scaler)   # this is our own f'n, above


SEPALWID
+++ Start of data-assembly for feature-regression! +++

+++ Scramble the data... +++

+++ 80/20 train-test split... +++

training with 112 rows;  testing with 29 rows

+++ Scale values... +++

                                                    input  ->  pred   des  
                                 [ 0.45  0.62  0.82 -0.73] ->    ?    2.8                  
                                 [ 1.05  1.13  1.75 -0.73] ->    ?    3.3                  
                                 [-1.   -1.42 -1.28  1.37] ->    ?    3.2                  
                                 [-0.75 -1.31 -1.28  1.37] ->    ?    3.4                  
                                 [-0.39  0.17  0.17 -0.73] ->    ?    2.3                  

                                                    input  ->  pred   des  
                                         [6.2 4.8 1.8 0. ] ->    ?    2.8                  
                                         [6.7 5.7 2.5 0. ] ->    ?    3.3                  
 

In [163]:
"""
~~~~~~~~~~~~
Petal length
~~~~~~~~~~~~

Here we set up for a regression model that will predict petal length  'petallen' (column index 2)

"""
print("PETALLEN")
print("+++ Start of data-assembly for feature-regression! +++\n")
# construct the correct X_all
X_all = np.concatenate( (A[:,0:2], A[:,3:]),axis=1)  # columns 0, 1, 3, and 4
y_all = A[:,2]                    # y (labels) ... is all of column 2, petallen
# print(f"y_all is \n {y_all}")
# print() 
# print(f"X_all (the features, first few rows) is \n {X_all[:5,:]}")
# print() 
# print() 



# Scramble the data, to give a different TRAIN/TEST split each time...
print("+++ Scramble the data... +++\n")
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
# print("target values to predict: \n",y_all)
# print("\nfeatures (a few)\n", X_all[0:5,:])



# We next separate into test data and training data ... 
print("+++ 80/20 train-test split... +++\n")
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

# print(f"Held-out data... (testing data: {len(y_test)})")
# print(f"y_test: {y_test}\n")
# print(f"X_test (few rows): {X_test[0:5,:]}")  # 5 rows
# print()
# print(f"Data used for modeling... (training data: {len(y_train)})")
# print(f"y_train: {y_train}\n")
# print(f"X_train (few rows): {X_train[0:5,:]}")  # 5 rows



# For NNets, it's important to keep the feature values near 0, say -1. to 1. or so
#    his is done through the "StandardScaler" in scikit-learn
print("+++ Scale values... +++\n")
from sklearn.preprocessing import StandardScaler

USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    scaler = StandardScaler()
    scaler.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scaler = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scaler.fit(X_train)  # still need to fit, though it does not change...

scaler   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Here are our scaled training and testing sets:

X_train_scaled = scaler.transform(X_train) # scale!
X_test_scaled = scaler.transform(X_test) # scale!

y_train_scaled = y_train  # the predicted/desired labels are not scaled
y_test_scaled = y_test  # not using the scaler

# reused from above - seeing the scaled data 
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],None)

# reused from above - seeing the unscaled data (inverting the scaler)
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],scaler)

# Train
# MLPRegressor predicts _floating-point_ outputs
from sklearn.neural_network import MLPRegressor

nn_regressor = MLPRegressor(hidden_layer_sizes=(6,7), 
                    max_iter=500,          # how many training epochs
                    verbose=True,          # do we want to watch as it trains?
                    shuffle=True,          # shuffle each epoch?
                    random_state=None,     # use for reproducibility
                    learning_rate_init=.1, # how much of each error to back-propagate
                    learning_rate = 'adaptive')  # how to handle the learning_rate

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_regressor.fit(X_train_scaled, y_train_scaled)
print("++++++++++  TRAINING:   end  +++++++++++++++")
print(f"The (squared) prediction error (the loss) is {nn_regressor.loss_:<6.3f}")
print(f"And, its square root:         {nn_regressor.loss_ ** 0.5:<6.3f}")
print()


#
# how did it do? Try out the TEST data...
#
print("PETALLEN TESTING RESULTS")
def ascii_table_for_regressor(Xsc,y,nn,scaler):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(Xsc) # all predictions
    Xpr = scaler.inverse_transform(Xsc)  # Xpr is the "X to print": unscaled data!
    # measure error
    error = 0.0
    # printing
    print(f"{'input ':>35s} ->  {'pred':^6s}  {'des.':^6s}   {'absdiff':^10s}") 
    for i in range(len(y)):
        pred = predictions[i]
        desired = y[i]
        result = abs(desired - pred)
        error += result
        # Xpr = Xsc   # if you'd like to see the scaled values
        print(f"{Xpr[i,:]!s:>35s} ->  {pred:<+6.2f}  {desired:<+6.2f}   {result:^10.2f}") 

    print("\n" + "+++++   +++++   +++++           ")
    print(f"average abs diff error:   {error/len(y):<6.3f}")
    print("+++++   +++++   +++++           ")
    
#
# let's see how it did on the test data (also the training data!)
#
ascii_table_for_regressor(X_test_scaled,
                          y_test_scaled,
                          nn_regressor,
                          scaler)   # this is our own f'n, above

PETALLEN
+++ Start of data-assembly for feature-regression! +++

+++ Scramble the data... +++

+++ 80/20 train-test split... +++

training with 112 rows;  testing with 29 rows

+++ Scale values... +++

                                                    input  ->  pred   des  
                                 [ 0.61 -0.79  0.92 -0.69] ->    ?    5.3                  
                                 [ 1.57  0.33  0.79 -0.69] ->    ?    6.0                  
                                 [-0.94  1.67 -1.22  1.45] ->    ?    1.5                  
                                 [-0.94  1.45 -1.09  1.45] ->    ?    1.5                  
                                 [-0.71  1.45 -1.36  1.45] ->    ?    1.5                  

                                                    input  ->  pred   des  
                                         [6.4 2.7 1.9 0. ] ->    ?    5.3                  
                                         [7.2 3.2 1.8 0. ] ->    ?    6.0                  
 

In [165]:
"""
~~~~~
Petal width
~~~~~

Here we set up for a regression model that will predict petal width   'petalwid' (column index 3)

"""
print("PETALWID")
print("+++ Start of data-assembly for feature-regression! +++\n")
# construct the correct X_all from the columns we want
# we use np.concatenate to combine parts of the dataset to get all-except-column 0:
#                     exclude 0  , include 1 to the end
X_all = np.concatenate( (A[:,0:3], A[:,4:]),axis=1)  # columns 0, 1, 2, and 4 


y_all = A[:,3]                    # y (labels) ... is all of column 0, sepallen (sepal length) 
#                                 # change the line above to make other columns the target (y_all)
# print(f"y_all is \n {y_all}")
# print() 
# print(f"X_all (the features, first few rows) is \n {X_all[:5,:]}")



# Scramble the data, to give a different TRAIN/TEST split each time...
print("+++ Scramble the data... +++\n")
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...
# print("target values to predict: \n",y_all)
# print("\nfeatures (a few)\n", X_all[0:5,:])



# We next separate into test data and training data ... 
print("+++ 80/20 train-test split... +++\n")
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

# print(f"Held-out data... (testing data: {len(y_test)})")
# print(f"y_test: {y_test}\n")
# print(f"X_test (few rows): {X_test[0:5,:]}")  # 5 rows
# print()
# print(f"Data used for modeling... (training data: {len(y_train)})")
# print(f"y_train: {y_train}\n")
# print(f"X_train (few rows): {X_train[0:5,:]}")  # 5 rows



# For NNets, it's important to keep the feature values near 0, say -1. to 1. or so
#    his is done through the "StandardScaler" in scikit-learn
print("+++ Scale values... +++\n")
from sklearn.preprocessing import StandardScaler

USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    scaler = StandardScaler()
    scaler.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
else:
    # this one does no scaling!  We still create it to be consistent:
    scaler = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
    scaler.fit(X_train)  # still need to fit, though it does not change...

scaler   # is now defined and ready to use...

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Here are our scaled training and testing sets:

X_train_scaled = scaler.transform(X_train) # scale!
X_test_scaled = scaler.transform(X_test) # scale!

y_train_scaled = y_train  # the predicted/desired labels are not scaled
y_test_scaled = y_test  # not using the scaler

# reused from above - seeing the scaled data 
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],None)

# reused from above - seeing the unscaled data (inverting the scaler)
ascii_table(X_train_scaled[0:5,:],y_train_scaled[0:5],scaler)

# Train
# MLPRegressor predicts _floating-point_ outputs
from sklearn.neural_network import MLPRegressor

nn_regressor = MLPRegressor(hidden_layer_sizes=(6,7), 
                    max_iter=500,          # how many training epochs
                    verbose=True,          # do we want to watch as it trains?
                    shuffle=True,          # shuffle each epoch?
                    random_state=None,     # use for reproducibility
                    learning_rate_init=.1, # how much of each error to back-propagate
                    learning_rate = 'adaptive')  # how to handle the learning_rate

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_regressor.fit(X_train_scaled, y_train_scaled)
print("++++++++++  TRAINING:   end  +++++++++++++++")
print(f"The (squared) prediction error (the loss) is {nn_regressor.loss_:<6.3f}")
print(f"And, its square root:         {nn_regressor.loss_ ** 0.5:<6.3f}")
print()

#
# how did it do? Try out the TEST data...
#
print("PETALWID TESTING RESULTS")
def ascii_table_for_regressor(Xsc,y,nn,scaler):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(Xsc) # all predictions
    Xpr = scaler.inverse_transform(Xsc)  # Xpr is the "X to print": unscaled data!
    # measure error
    error = 0.0
    # printing
    print(f"{'input ':>35s} ->  {'pred':^6s}  {'des.':^6s}   {'absdiff':^10s}") 
    for i in range(len(y)):
        pred = predictions[i]
        desired = y[i]
        result = abs(desired - pred)
        error += result
        # Xpr = Xsc   # if you'd like to see the scaled values
        print(f"{Xpr[i,:]!s:>35s} ->  {pred:<+6.2f}  {desired:<+6.2f}   {result:^10.2f}") 

    print("\n" + "+++++   +++++   +++++           ")
    print(f"average abs diff error:   {error/len(y):<6.3f}")
    print("+++++   +++++   +++++           ")
    
#
# let's see how it did on the test data (also the training data!)
#
ascii_table_for_regressor(X_test_scaled,
                          y_test_scaled,
                          nn_regressor,
                          scaler)   # this is our own f'n, above

PETALWID
+++ Start of data-assembly for feature-regression! +++

+++ Scramble the data... +++

+++ 80/20 train-test split... +++

training with 112 rows;  testing with 29 rows

+++ Scale values... +++

                                                    input  ->  pred   des  
                                 [-1.74 -0.16 -1.39  1.4 ] ->    ?    0.2                  
                                 [-0.91  1.77 -1.28  1.4 ] ->    ?    0.3                  
                                 [-0.19 -1.13 -0.15 -0.72] ->    ?    1.0                  
                                 [ 1.25  0.08  0.75 -0.72] ->    ?    2.3                  
                                 [ 1.73 -0.4   1.42 -0.72] ->    ?    1.8                  

                                                    input  ->  pred   des  
                                         [4.4 3.  1.3 1. ] ->    ?    0.2                  
                                         [5.1 3.8 1.5 1. ] ->    ?    0.3                  
 

#### Average absolute difference errors:
+ Sepal length: 0.225
+ Sepal width: 0.251
+ Petal length: 0.166
+ Petal width: 0.145

This data implies that petal width is the most predictable, while sepal width is the least. I can't say I know anything about flowers, so maybe this makes sense!